<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/temp_hosting/rel_strength.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import yfinance as yf
# Example stock data (replace with your actual data)
nifty50_url = "https://raw.githubusercontent.com/anirbanghoshsbi/data/main/ind_nifty50list.csv"
nifty50 = pd.read_csv(nifty50_url)
tickers = nifty50['Symbol'].apply(lambda x: x + ".NS").tolist()
# Bank Nifty Stocks
nifty_stocks = tickers
start_date = '2023-01-02'

def download_data(stock_list,start_date):
    data = yf.download(stock_list, start=start_date)
    data = data[['Adj Close']]
    return data

# Example benchmark index data (replace with your actual data)
benchmark_index = {
    'Index': ['MyIndex'],
    'Value': [2000]
}

# Create DataFrames
stock_df = download_data(nifty_stocks, start_date)
benchmark_df = yf.download('^nsei',start_date)

# Assuming stock_data is your DataFrame with MultiIndex columns

# Reset the index to flatten it
stock_df.reset_index(inplace=True)

# Reset the columns index to flatten the MultiIndex
stock_df.columns = stock_df.columns.get_level_values(1)

# Convert index to datetime
stock_df.columns.values[0]='Date'

# Calculate percentage change for each stock
percentage_change_data = stock_df.set_index('Date').pct_change(90)*100

# Create a new DataFrame to store the percentage change data
percentage_change_df = pd.DataFrame(percentage_change_data)





benchmark_df['Percentage Change'] = benchmark_df['Adj Close'].pct_change(90) * 100

# Remove NaN values
stock_df.dropna(inplace=True)
benchmark_df.dropna(inplace=True)



[*********************100%%**********************]  50 of 50 completed
[*********************100%%**********************]  1 of 1 completed


In [94]:
merged_df = percentage_change_df.join(benchmark_df['Percentage Change'], how='inner')

In [95]:
covariance = merged_df.cov()['Percentage Change']
benchmark_variance = np.var(benchmark_df['Percentage Change'])
beta = covariance / benchmark_variance

In [96]:
# Get the last row of stock_df
last_row = stock_df.iloc[-1]  # or stock_df.tail(1)

# Create a new DataFrame with the last row and the beta series
new_df = pd.DataFrame({
    'Percentage_change': last_row.iloc[1:51],
    'Beta': beta,
    'volatility':np.std(percentage_change_df),


})



In [97]:
# Reset the index of the new DataFrame
new_df = new_df.dropna().reset_index()


In [98]:
new_df['Relative_Strength']=new_df['Percentage_change']/benchmark_df['Percentage Change'].iloc[-1]

In [102]:
# Optional: Clustering
X = new_df[['Relative_Strength', 'volatility','Beta','Percentage_change']].values
kmeans = KMeans(n_clusters=10)
kmeans.fit(X)
new_df['Cluster'] = kmeans.labels_

# Print clustering results
print(new_df[['index', 'Cluster']])

            index  Cluster
0     ADANIENT.NS        9
1   ADANIPORTS.NS        8
2   APOLLOHOSP.NS        1
3   ASIANPAINT.NS        9
4     AXISBANK.NS        0
5   BAJAJ-AUTO.NS        3
6   BAJAJFINSV.NS        8
7   BAJFINANCE.NS        1
8   BHARTIARTL.NS        8
9         BPCL.NS        5
10   BRITANNIA.NS        7
11       CIPLA.NS        8
12   COALINDIA.NS        5
13    DIVISLAB.NS        2
14     DRREDDY.NS        1
15   EICHERMOT.NS        7
16      GRASIM.NS        6
17     HCLTECH.NS        8
18    HDFCBANK.NS        8
19    HDFCLIFE.NS        5
20  HEROMOTOCO.NS        7
21    HINDALCO.NS        5
22  HINDUNILVR.NS        6
23   ICICIBANK.NS        0
24  INDUSINDBK.NS        8
25        INFY.NS        8
26         ITC.NS        5
27    JSWSTEEL.NS        0
28   KOTAKBANK.NS        8
29          LT.NS        2
30        LTIM.NS        7
31         M&M.NS        6
32      MARUTI.NS        4
33   NESTLEIND.NS        6
34        NTPC.NS        5
35        ONGC.NS        5
3

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [103]:
df_cluster_3 = new_df[new_df['Cluster'] == 0]

In [104]:
df_cluster_3

,index,Percentage_change,Beta,volatility,Relative_Strength,Cluster
4,AXISBANK.NS,1130.300049,0.598768,6.195021,156.126482,0
23,ICICIBANK.NS,1107.900024,0.986603,6.037768,153.032404,0
27,JSWSTEEL.NS,887.25,1.238498,7.375658,122.554379,0
39,SBIN.NS,801.299988,2.167028,12.183015,110.682246,0
41,TATACONSUM.NS,1102.650024,1.636761,9.189152,152.30723,0
42,TATAMOTORS.NS,999.5,2.464593,13.852915,138.059287,0
